In [10]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [11]:
import torch
torch.set_float32_matmul_precision('medium')

In [12]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40

path = "../data/ModelNet40"
categories = ['airplane']

tr, te = get_dataloaders(path, categories=categories)

Loading renders for airplane: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3391.09it/s]


In [13]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 192, 192, 256],
    "num_layers": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192, 128, 64, 32],
        "num_layers": 1,
        "attn_heads": (8, 8, None, None, None),
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

model = models.DiffusionBase(model, lr=1e-4)

In [14]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 192, 192, 256],
    "num_layers": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192],
        "num_layers": 1,
        "attn_heads": 8,
    },
    {
        "features": [192, 128, 64, 32],
        "num_layers": 1,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

model = models.DiffusionBase(model, lr=1e-4)

In [15]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_name = "-".join(categories) if categories else "all"
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

epochs = 400
trainer = L.Trainer(
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 32.6 M | train
------------------------------------------
32.6 M    Trainable params
0         Non-trainable params
32.6 M    Total params
130.378   Total estimated model params size (MB)
342       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [17]:
import torch

torch.save(model.state_dict(), "../checkpoints/ModelNet40/airplane-good")

In [18]:
model = model.cuda().eval()

from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

In [19]:
from utils.visualization import visualize_notebook

preds = ddpm_sched.sample(model, 16, 2048)
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)


%|                                                                                                                                                                               | 0/1000 [00:00<?, ?it/s]
%|▎                                                                                                                                                                      | 2/1000 [00:00<01:08, 14.58it/s]
%|▋                                                                                                                                                                      | 4/1000 [00:00<01:05, 15.15it/s]
%|█                                                                                                                                                                      | 6/1000 [00:00<01:04, 15.34it/s]
%|█▎                                                                                                                                                                     | 8/1000 [00:00<01

Output()